[View in Colaboratory](https://colab.research.google.com/github/riquebueno/topicosaprendizadomaquina/blob/master/Trabalho1TopicosAprendMaq.ipynb)

In [1]:
print("alo mundo")

alo mundo


In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0
